In [3]:
%%capture
!pip install chromadb
!pip install beautifulsoup4
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install langchainhub
!pip install sentence-transformers
!pip install langchain-openai

## 사용자 별 vector db 데이터 따로 사용 테스트

In [4]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# url로 가져오기
loader = WebBaseLoader("https://www.scourt.go.kr/portal/news/NewsViewAction.work?pageIndex=1&searchWord=&searchOption=&seqnum=9782&gubun=4&type=5")

# 데이터 로딩하기
data = loader.load()

# 데이터 전처리 필요
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
docs = loader.load_and_split(text_splitter)

In [5]:
import chromadb
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import uuid
import random

stf_embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# ChromaDB의 PersistentClient를 생성합니다.
persistent_client = chromadb.PersistentClient()
# "my_chroma_collection"이라는 이름의 컬렉션을 가져오거나 생성합니다.
collection = persistent_client.get_or_create_collection("chroma_collection")

for i, doc in enumerate(docs):
    doc.metadata["chunk_id"] = i  # Chunk ID 추가
    # 무작위로 임의의 페이지 번호를 삽입합니다.
    doc.metadata["page_number"] = random.randint(0, 5)
    collection.add(
        ids=[str(uuid.uuid1())],
        metadatas=doc.metadata,
        documents=doc.page_content,
    )

c:\gitlist\transform_text\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Chroma 객체를 생성합니다.
langchain_chroma = Chroma(
    # PersistentClient를 전달합니다.
    client=persistent_client,
    # 사용할 컬렉션의 이름을 지정합니다.
    collection_name="chroma_collection",
    # 임베딩 함수를 전달합니다.
    embedding_function=stf_embeddings,
)

# 컬렉션의 항목 수를 출력합니다.
print(
    "현재 저장된 Collection 의 개수는 ",
    langchain_chroma._collection.count(),
    " 개 입니다.",
)

현재 저장된 Collection 의 개수는  51  개 입니다.


In [9]:
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 를 사용
openai_embeddings = OpenAIEmbeddings()
# client 생성
new_client = chromadb.EphemeralClient()


# OpenAI 임베딩과 Chroma 클라이언트를 사용하여 문서에서 Chroma 검색 클라이언트를 생성합니다.
openai_lc_client = Chroma.from_documents(
    docs, openai_embeddings, client=new_client, collection_name="chroma_collection"
)

query = "채무자 관련 법에 대해 설명해줘."
# 질의를 사용하여 유사도 검색을 수행합니다.
docs = openai_lc_client.similarity_search(query)
# 검색 결과에서 첫 번째 문서의 내용을 출력합니다.
print(docs[0].page_content)

2. 채무자회생법 제564조 제1항 제5호는 ‘채무자가 이 법에 정하는 채무자의 의무를 위반한 때’를 면책불허가사유로 규정하고 있다. 지급불능 상태에 빠진 채무자에 대하여 경제적 재기와 갱생의 기회를 부여하여야 한다는 면책제도의 이념, 특별한 사정이 없으면 원칙적으로 면책을 허가하여야 한다고 규정하고 있는 채무자회생법 제564조 제1항의 입법취지, 파산선고를 받았음에도 면책이 불허가되는 경우 채무자가 결과적으로 입게 되는 신분상 불이익 등을 고려하면, 위 조항에서 일반적·보충적으로 규정하고 있는 ‘이 법에 정하는 채무자의 의무를 위반한 때’라 함은 그 의무 위반의 대상이나 정도를 엄격하게 제한적으로 해석할 필요가 있다.


## 결과 테스트

In [8]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [10]:
from langchain.agents.agent_toolkits import create_retriever_tool

retriever = langchain_chroma.as_retriever() # 리트리버 초기화

tool = create_retriever_tool(
retriever,
"cusomter_service",
"Searches and returns documents regarding the customer service guide.",
)

tools = [tool]

In [11]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI()

memory_key = "history"

from langchain.agents.openai_functions_agent.agent_token_buffer_memory import (
    AgentTokenBufferMemory,
)

memory = AgentTokenBufferMemory(memory_key=memory_key, llm=llm)

c:\gitlist\transform_text\.conda\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [12]:
# LLM 셋업하기 

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder

# LLM이 어떤 행동을 해야하는지 알려주는 프롬프트 
system_message = SystemMessage(
    content=(
        "You are a nice customer service agent."
        "Do your best to answer the questions."
        "Feel free to use any tools available to look up "
        "You provide a service that provides answers to questions"
        "relevant information, only if necessary"
        # 할루시네이션.
        "If you don't know the answer, just say that you don't know. Don't try to make up an answer."
        # 한국어
        "Make sure to answer in Korean."
    )
)

# 위에서 만든 프롬프트를 바탕으로 LLM에게 줄 프롬프트 형식 셋업 
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)],
)

# 위에서 만든 llm, tools, prompt를 바탕으로 에이전트 만들어주기 
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

c:\gitlist\transform_text\.conda\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.openai_functions_agent.base.OpenAIFunctionsAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_openai_functions_agent instead.
  warn_deprecated(


In [13]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
agent=agent,
tools=tools,
memory=memory,
verbose=True,
return_intermediate_steps=True,
)

result = agent_executor({"input": "안녕하세요"})
print(result)

c:\gitlist\transform_text\.conda\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
안녕하세요! 궁금하신 사항이 있으신가요? 무엇을 도와드릴까요?

> Finished chain.
{'input': '안녕하세요', 'history': [HumanMessage(content='안녕하세요'), AIMessage(content='안녕하세요! 궁금하신 사항이 있으신가요? 무엇을 도와드릴까요?')], 'output': '안녕하세요! 궁금하신 사항이 있으신가요? 무엇을 도와드릴까요?', 'intermediate_steps': []}


In [16]:
result = agent_executor({"input": "채무자 법에 대해 알려줘"})
print()
print(result)



> Entering new AgentExecutor chain...

Invoking: `cusomter_service` with `{'query': '채무자 법'}`


1. 채무자회생법 제564조 제1항 제1호, 제658조, 제321조에 따르면, 채무자는 파산관재인·감사위원 또는 채권자집회(이하 ‘파산관재인 등’이라 한다)의 요청이 있으면 ‘파산에 관하여 필요한 설명’을 하여야 하고, 채무자가 정당한 사유 없이 설명을 하지 않거나 허위의 설명을 한 때에는 설명의무위반죄로 처벌하며, 채무자에게 설명의무위반죄에 해당하는 행위가 인정되면 면책불허가사유에 해당한다.

1. 채무자회생법 제564조 제1항 제1호, 제658조, 제321조에 따르면, 채무자는 파산관재인·감사위원 또는 채권자집회(이하 ‘파산관재인 등’이라 한다)의 요청이 있으면 ‘파산에 관하여 필요한 설명’을 하여야 하고, 채무자가 정당한 사유 없이 설명을 하지 않거나 허위의 설명을 한 때에는 설명의무위반죄로 처벌하며, 채무자에게 설명의무위반죄에 해당하는 행위가 인정되면 면책불허가사유에 해당한다.

1. 채무자회생법 제564조 제1항 제1호, 제658조, 제321조에 따르면, 채무자는 파산관재인·감사위원 또는 채권자집회(이하 ‘파산관재인 등’이라 한다)의 요청이 있으면 ‘파산에 관하여 필요한 설명’을 하여야 하고, 채무자가 정당한 사유 없이 설명을 하지 않거나 허위의 설명을 한 때에는 설명의무위반죄로 처벌하며, 채무자에게 설명의무위반죄에 해당하는 행위가 인정되면 면책불허가사유에 해당한다.

☞  채무자는 1942년생으로 아들의 거주지에서 채무자와 배우자의 노령연금 및 아들의 수입에 의존하여 생활한다는 내용을 기재한 파산·면책신청서류를 제출하였고, 파산관재인은 채무자에게 “① 채무자의 3년간 직업이력과 소득 설명, ② 가계수지표를 현실적으로 작성하여 제출, ③ 자녀의 소득자료, 생활비 통장내역 제출, ④ 채무자와 배우자의 보험가입조회서 제출, ⑤ 채무자의 자녀들에게 과세된 다수의 부동산에 관한 등기부등